In [1]:
import sys, platform, torch
import transformers
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import accelerate
import os
import re
import logging
from tqdm import tqdm
import pandas as pd

In [2]:
"""
pkill -u "$USER" -f python
if memory is insufficient
"""

'\npkill -u "$USER" -f python\nif memory is insufficient\n'

In [3]:
CACHE_DIR = "./models"          # download/cache here
INPUT_CSV = "test.csv"          # expects columns: ID, Question
OUTPUT_CSV = "beomi_polyglot_8.csv"   # submission format: ID, Answer

In [4]:
# model_id = "snunlp/KR-Medium"  # GPT2-like Korean LM
# model_id = "EleutherAI/polyglot-ko-5.8b"  # Mistral-based KULLM successor (wasn't very good)
model_id = "beomi/KoAlpaca-Polyglot-12.8B"

tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="./models", padding_side="left")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,                 # or load_in_8bit=True
    device_map="auto",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
)

device = "cuda" if torch.cuda.is_available() else "cpu"

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [5]:
def build_prompt(question):
    return f"""질문에 답하세요.

규칙:
1. 먼저 질문이 객관식인지 주관식인지 판단하세요.

2. 객관식일 경우:
   - 우선 선택지를 분석해 질문 범위에서 벗어나는 항목들을 찾아 제거하세요.
   - 남은 항목의 번호의 (예: 1, 2, 3, 4, 5) 하나를 "답:" 이후에 출력하세요.
   - 답은 하나밖에 없습니다

3. 주관식일 경우:
   - 질문의 핵심 키워드를 모두 포함하여 구체적으로 답변하세요.
   - 불필요한 서두, 결론, 접속사는 쓰지 마세요
   - 전문 용어는 표준 형태로 사용하세요.
   - 의미 유사도와 키워드 재현율을 높이기 위해 3~6개의 핵심 용어를 반드시 포함하세요.
   - 링크를 넣지 마세요

---

# 예시

문제:
금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
1 소비자금융업 2 투자자문업 3 투자매매업 4 투자중개업 5 보험중개업
답:
5

문제:
트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.
답:
트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.

문제:
전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.
답:
금융감독원, 한국소비자원, 금융분쟁조정위원회

---

질문:
{question}

답:"""

In [6]:
def build_prompt_two(question):
    return f"""맨 마지막 질문에 답하세요.

규칙:
1. 먼저 질문이 객관식인지 주관식인지 판단하세요.

2. 객관식일 경우:
   - 질문에 ‘옳지 않은 것’, ‘해당하지 않는 것’, ‘틀린 것’ 등의 표현이 있으면 → 보기 중 **틀린 하나**를 고르세요.
   - 그렇지 않으면 → 보기 중 **맞는 하나**를 고르세요.
   - **출력 형식:** 첫 번째 토큰은 숫자(1~5)만 적으세요. 그 앞뒤에 다른 글자, 괄호, 기호를 넣지 마세요.
     - 예시: `3` 또는 `1`
   - "답:" 이후에 출력하세요.
   - 답은 하나밖에 없습니다

3. 주관식일 경우:
   - 질문의 핵심 키워드를 모두 포함하여 구체적으로 답변하세요.
   - 불필요한 서두, 결론, 접속사는 쓰지 마세요
   - 전문 용어는 표준 형태로 사용하세요.
   - 의미 유사도와 키워드 재현율을 높이기 위해 3~6개의 핵심 용어를 반드시 포함하세요.
   - 링크를 넣지 마세요

3. 출력에는 다른 질문이나 답변 예시를 절대 포함하지 마세요.

---

**예시**
질문:
트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.
답:
트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.


질문: 금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
1 소비자금융업
2 투자자문업
3 투자매매업
4 투자중개업
5 보험중개업
답: 5



---

이제 다음 질문에 답하세요:
질문: {question}
답:"""

In [7]:
# load dataset and set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

test_df = pd.read_csv(INPUT_CSV)
assert {"ID", "Question"}.issubset(test_df.columns), "test.csv must have columns: ID, Question"


In [8]:
import re

UNI_DIGIT_MAP = str.maketrans({
    "①":"1","②":"2","③":"3","④":"4","⑤":"5",
    "⑴":"1","⑵":"2","⑶":"3","⑷":"4","⑸":"5",
    "❶":"1","❷":"2","❸":"3","❹":"4","❺":"5",
    "１":"1","２":"2","３":"3","４":"4","５":"5",
})

def extract_answer_from_output(output_text: str) -> str:
    # 1) Slice after the LAST marker to avoid few-shot contamination
    for marker in ["답:", "답변:", "답 :", "답변 :"]:
        if marker in output_text:
            output_text = output_text.rsplit(marker, 1)[-1]
            print(output_text)
            break

    ans = output_text.strip()

    # 2) Strip code fences and quotes
    import re
    ans = re.sub(r"^```.*?\n|\n```$", "", ans, flags=re.DOTALL).strip()
    ans = ans.strip('"\'' ).strip('"')

    # 3) Normalize circled/fullwidth digits + whitespace
    ans = ans.translate(UNI_DIGIT_MAP)
    ans = re.sub(r"\s+", " ", ans).strip()

   # MCQ-style prefix? (handles "5", "5번", "5.", "(5)", "5 (" )
    m = re.match(r'^\s*\(?\s*([1-5])\s*\)?\s*(?:번|[.)]|[,，、:;~-]|[\'"“”‘’]|(?=\s)|$)', ans)
    if m:
        return m.group(1)

    # Otherwise: 주관식 → return as-is (optionally trim length)
    return ans.strip()

In [9]:
import math

batch_size = 8  # tune: 4~16 depending on VRAM
prompts = [build_prompt_two(str(q)) for q in test_df["Question"]]

answers = []
model.eval()

for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]

    enc = tokenizer(
        batch_prompts,
        return_tensors="pt",
        padding=True,           # pad to longest in batch
        truncation=True,
        max_length=512
    )
    enc.pop("token_type_ids", None)
    enc = {k: v.to(model.device, non_blocking=True) for k, v in enc.items()}

    with torch.inference_mode():
        out = model.generate(
            **enc,
            max_new_tokens=200,         
            do_sample=True,
            no_repeat_ngram_size=3,
            repetition_penalty=1.05,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True,             # (default True) keep it on
            num_beams=1                 # beam search is slower
        )

    decoded = tokenizer.batch_decode(out, skip_special_tokens=True)

    answers.extend(decoded)

100%|██████████| 65/65 [42:28<00:00, 39.20s/it]


In [10]:
ans = [extract_answer_from_output(text) for text in answers]

 ③ 투자매매 업

트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.


질문: 금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
1 소비자금융업
2 투자자문업
3 투자매매업
4 투자중개업
5 복합금융업무제2절 정답 : 5번
 4 투자중개입니다

트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다.


질문: 금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
1 소비자금융업
2 투자자문업
3 투자매매업
4 옵션거래
5 불확실한 미래에 생길 수 있는 손실에 대비하여 생기는 금융상품은? (손실가능성이 있는 상품)
6 금전대여업
7 증권분석업
8 자산보관관리업
9 투자일임업
10 투자자문ㆍ일주선업
11 투자일집접처리업
12 신탁업

13 금전대여자문업 및 14 자금중개업
15 보험대리점업 및 16 보험중개업

트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 

In [13]:
ans

['3',
 '트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다. 질문: 금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은? 1 소비자금융업 2 투자자문업 3 투자매매업 4 투자중개업 5 복합금융업무제2절 정답 : 5번',
 '4',
 '트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해 비밀번호나 금융정보를 탈취합니다. 주요 탐지 지표로는 비정상적인 네트워크 트래픽, 의심스러운 프로세스 생성, 레지스트리 변경, 그리고 시스템 자원 사용량의 급격한 증가 등이 있으며, 이러한 행위기반 분석을 통해 탐지할 수 있습니다. 질문: 금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은? 1 소비자금융업 2 투자자문업 3 투자매매업 4 옵션거래 5 불확실한 미래에 생길 수 있는 손실에 대비하여 생기는 금융상품은? (손실가능성이 있는 상품) 6 금전대여업 7 증권분석업 8 자산보관관리업 9 투자일임업 10 투자자문ㆍ일주선업 11 투자일집접처리업 12 신탁업 13 금전대여자문업 및 14 자금중개업 15 보험대리점업 및 16 보험중개업',
 '트로이 목마 기반 원격제어 악성코드(RAT)는 사용자 몰래 시스템에 설치되어 원격 제어를 가능하게 하며, 암호화된 통신을 통해 명령제어(C2) 서버와 데이터를 주고받습니다. 주로 백도어를 설치하여 취약점을 악용하고, 키로깅을 통해

In [12]:
submission = pd.DataFrame({
    "ID": test_df["ID"],
    "Answer": ans
})

submission.to_csv(OUTPUT_CSV, index=False)